# Pembuatan Komponen Pengolahan Data dalam Machine Learning Pipeline

## Data Ingestion

In [2]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration import pipeline

import os
import sys
import pprint
import tempfile
import urllib
pp = pprint.PrettyPrinter()

### Set Variable

In [3]:
PIPELINE_NAME = "sarcasm-pipeline"
SCHEMA_PIPELINE_NAME = "sarcasm-tfdv-schema"


#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [4]:
# sys.path.append(".")
os.getcwd()

'e:\\[Coding]\\Learning\\ML-Pipeline'

### ExampleGen

In [5]:
DATA_ROOT = 'data'

In [6]:
context = InteractiveContext()

In [7]:
# output = example_gen_pb2.Output(
#     split_config = example_gen_pb2.SplitConfig(splits=[
#         example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
#         example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
#     ])
# )

In [8]:
# example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [9]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)

In [10]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [11]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] C:\Users\Jals\AppData\Local\Temp\tfx-interactive-2025-02-06T20_53_05.502429-dmjl18yu\CsvExampleGen\examples\1


In [12]:
# # Get the URI of the output artifact representing the training examples, which is a directory
# train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# # Get the list of files in this directory (all compressed TFRecord files)
# tfrecord_filenames = [os.path.join(train_uri, name)
#                       for name in os.listdir(train_uri)]

# # Create a `TFRecordDataset` to read these files
# dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# # Iterate over the first 3 records and decode them.
# for tfrecord in dataset.take(3):
#   serialized_example = tfrecord.numpy()
#   example = tf.train.Example()
#   example.ParseFromString(serialized_example)
#   pp.pprint(example)

## Tahapan Data Validation

### Statistic Gen
Membuat Summary Statistic menggunakan komponen StatisticsGen(). Komponen memiliki parameter input berupa examples untuk menerima dataset dari komponen ExampleGen

In [13]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

In [14]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [15]:
context.show(statistics_gen.outputs["statistics"])

### Data Schema
Setelah membuat summary statistic, kita perlu membuat data schema. Pada latihan ini, kita menggunakan komponen ShcemaGen() untuk membuat data schema. Komponen ini akan menerima input berupa summary statistic.

In [16]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"])
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [17]:
context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'headline',BYTES,required,,-
'is_sarcastic',INT,required,,-


Berdasarkan tampilan data schema di atas, diketahui bahwa dataset yang kita gunakan terdiri dari dua fitur, yaitu “headline” dan “is_sarcastic”. Fitur pertama memiliki tipe bytes, sedangkan fitur kedua memiliki memiliki tipe integer. Selain itu, berdasarkan data schema di atas kedua fitur tersebut haruslah lengkap untuk setiap dataset.

### Indentify Anomali in Dataset

In [18]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

In [19]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [20]:
context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

Setelah melakukan data validation, tahap berikutnya adalah melakukan data preprocessing. Tujuan dari tahap ini adalah mengubah data mentah menjadi data yang siap digunakan untuk melatih model. Pada latihan ini, kita menggunakan TFT dan komponen Transform untuk melakukan data preprocessing.

In [21]:
TRANSFORM_MODULE_FILE = "sarcasm_transform.py"

In [22]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf

LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Overwriting sarcasm_transform.py


In [23]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

In [24]:
pip install wheel setuptools

Note: you may need to restart the kernel to use updated packages.


In [27]:
# context.run(transform)

In [ ]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None,
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None,
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None,
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None,
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}